In [6]:
import controlVariables as var
import numpy as np
import math

class InverseKinematics():
    def __init__(self, FR_coord, FL_coord, BR_coord, BL_coord, eualarAng):
        self.FR_coord = FR_coord
        self.FL_coord = FL_coord
        self.BR_coord = BR_coord
        self.BL_coord = BL_coord
        self.roll = eualarAng[0]
        self.pitch = eualarAng[1]
        self.yaw = eualarAng[2]


    def rotMat(self):
        M11 = np.cos(self.pitch)*np.cos(self.yaw)
        M12 = np.sin(self.roll)*np.sin(self.pitch)*np.cos(self.yaw) - np.cos(self.roll)*np.sin(self.yaw)
        M13 = np.cos(self.roll)*np.sin(self.pitch)*np.cos(self.yaw) + np.sin(self.roll)*np.sin(self.yaw)
        M21 = np.cos(self.pitch)*np.sin(self.yaw)
        M22 = np.sin(self.roll)*np.sin(self.pitch)*np.sin(self.yaw) + np.cos(self.roll)*np.cos(self.yaw)
        M23 = np.cos(self.roll)*np.sin(self.pitch)*np.sin(self.yaw) - np.sin(self.roll)*np.cos(self.yaw)
        M31 = -np.sin(self.pitch)
        M32 = np.sin(self.roll)*np.cos(self.pitch)
        M33 = np.cos(self.roll)*np.cos(self.pitch)
        return np.array([
                        [M11, M12, M13],
                        [M21, M22, M23],
                        [M31, M32, M33]
                        ])




In [2]:
"""        ^ X
           |
 FL *------|-----* FR
    |      |     |  BODY_LENGTH
    |      |     |   300mm
    |      ------|---------> Y
    |            |
    |            |
    |            |
 BL *------------* BR
      BODY WIDTH
        172mm


"""

'        ^ X\n           |\n FL *------|-----* FR\n    |      |     |  BODY_LENGTH\n    |      |     |   300mm\n    |      ------|---------> Y\n    |            |\n    |            |\n    |            |\n BL *------------* BR\n      BODY WIDTH\n        172mm\n\n\n'

In [8]:
import numpy as np
BODY_LENGTH = 300
BODY_WIDTH  = 172

FR = np.array([0, 104, 200])
FL = np.array([0, 104, 200])
BR = np.array([0, 104, 200])
BL = np.array([0, 104, 200])
eularAng = [0,  0, 0]
eularAng = [np.deg2rad(eularAng[0]), np.deg2rad(eularAng[1]), np.deg2rad(eularAng[2])]

IK = InverseKinematics(FR,FL,BR,BL, eularAng)

np.dot(FR, IK.rotMat())

array([  0., 104., 200.])

In [9]:

joint_ang = np.zeros([3])

M_R = np.array([1, 1, 1])
M_L = np.array([1, -1, 1])
M_F = np.array([1, 1, 1])
M_B = np.array([-1, 1, 1])
BODY_SCALE = np.array([BODY_LENGTH/2, BODY_WIDTH/2, 0])

In [10]:
translate_FR = BODY_SCALE*M_F*M_R
IK.FR_coord = (np.dot((IK.FR_coord * M_R + translate_FR), IK.rotMat()) - translate_FR) *M_R

translate_FL = BODY_SCALE*M_F*M_L
IK.FL_coord = (np.dot((IK.FL_coord * M_L + translate_FL), IK.rotMat()) - translate_FL)*M_L

translate_BR = BODY_SCALE*M_B*M_R
IK.BR_coord = (np.dot((IK.BR_coord * M_R + translate_BR), IK.rotMat()) - translate_BR) *M_R

translate_BL = BODY_SCALE*M_B*M_L
IK.BL_coord = (np.dot((IK.BL_coord * M_L + translate_BL), IK.rotMat()) - translate_BL) *M_L



In [11]:

print(IK.FR_coord, '\n', IK.FL_coord, '\n', IK.BR_coord, '\n', IK.BL_coord)


[  0. 104. 200.] 
 [  0. 104. 200.] 
 [  0. 104. 200.] 
 [  0. 104. 200.]


In [15]:
def get_joint_angle(coord):
    joint_ang = np.zeros([3])
    r = np.sqrt(coord[0]**2 + coord[1]**2 + coord[2]**2)
    print('r=', r, '  max r=', var.MAX_LEG_R)
    if r < var.MIN_LEG_R or r > var.MAX_LEG_R:
        pass
    else:
        r_yz = np.sqrt(coord[1]**2 + coord[2]**2) 
        alpha = np.arccos(coord[1]/r_yz)
        th0 = np.arccos(var.L1/r_yz) - alpha #phi1 - phi2
        if not np.isnan(th0):
            var.JointAngles.target_angle[0] = th0
            joint_ang[0] = th0

        a = 2*var.L2*r_yz*np.sin(var.JointAngles.target_angle[0] + alpha)
        b = 2*var.L2*coord[0]
        c = coord[0]**2 + var.L2**2 - var.L3**2 + (a/(2*var.L2))**2
        d = np.sqrt(a**2 + b**2)
        beta = np.arccos(a/d)

        if (coord[0] < 0 ):
            beta = -beta
        if a/d > 1 or a/d < -1 or c/d >1 or c/d < -1:
            pass
        else:
            th1 = beta + np.arcsin(c/d)
            if not np.isnan(th1):
                var.JointAngles.target_angle[1] = th1
                joint_ang[1] = th1
            
            th2 = np.arccos((coord[0] + var.L2*np.cos(var.JointAngles.target_angle[1]))/var.L3)
            if not np.isnan(th2):
                var.JointAngles.target_angle[2] = th2
                joint_ang[2] = th2
    return joint_ang





In [16]:
get_joint_angle(IK.FR_coord)

r= 225.42404485768594   max r= 316.4369587857104


array([0.        , 0.72972766, 0.72972766])

In [17]:
from InverseKinematics import InverseKinematics 
IK = InverseKinematics()

In [44]:
FR_coord = np.array([0, 104, 220])
FL_coord = np.array([0, 104, 220])
BR_coord = np.array([0, 104, 220])
BL_coord = np.array([0, 104, 220])
eularAng = [0, 0.1, 0]


IK.get_FR_joint_angles(FR_coord, eularAng)

array([0.        , 0.80307192, 0.9966936 ])

In [45]:
IK.get_FL_joint_angles(FL_coord, eularAng)

array([0.        , 0.80307192, 0.9966936 ])

In [46]:
IK.get_BR_joint_angles(BR_coord, eularAng)

array([0.        , 0.64878379, 0.85609385])

In [47]:
IK.get_BL_joint_angles(BL_coord, eularAng)

array([0.        , 0.64878379, 0.85609385])

In [1]:
IK.MAX_LEG_R

NameError: name 'IK' is not defined

In [1]:
from InverseKinematics import InverseKinematics
import numpy as np
IK = InverseKinematics()
IK.MAX_LEG_R

316.4369587857104

In [2]:
IK.MIN_LEG_R

129.78773759373516

In [19]:
coord = [0,204,280]
r = np.sqrt([coord[0]**2 + coord[1]**2 + coord[2]**2])
r

array([346.43325475])

In [20]:
IK.is_singularity(coord)

True

In [30]:
import rclpy
rclpy.init()
node = rclpy.create_node("test1")
rclpy.shutdown()

In [29]:
rclpy.shutdown()

In [31]:
node.destroy_node()

In [29]:
a = [0,0,1,0]
np.any(a) 

True

In [68]:
class CMD():
    a =1
    b=2
    c=3


class cmd_print():
    def __init__(self, cmd):
        self.cmd = cmd

    def print(self):
        print(self.cmd.c)
        

In [69]:
prnt = cmd_print(CMD)

In [70]:
prnt.print()

3


In [30]:
a[1] = True

In [31]:
a

[0, True, 1, 0]